# Generates the table of the ontological issues.

### Note this code assumes that you've already computed psx -- the predicted probabilities for all examples in the training set using four-fold cross-validation. If you have no done that you will need to use `imagenet_train_crossval.py` to do this!

In [4]:
# These imports enhance Python2/3 compatibility.
from __future__ import print_function, absolute_import, division, unicode_literals, with_statement

In [9]:
import cleanlab
import numpy as np
import torch

# For visualizing images of label errors
from PIL import Image
from torchvision import datasets
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix

# urllib2 for python2 and python3
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen
    
# where imagenet dataset is located
train_dir = '/datasets/datasets/imagenet/val/'

In [6]:
# Set-up name mapping for ImageNet train data
url = 'https://gist.githubusercontent.com/aaronpolhamus/964a4411c0906315deb9f4a3723aac57/'
url += 'raw/aa66dd9dbf6b56649fa3fab83659b2acbf3cbfd1/map_clsloc.txt'
with urlopen(url) as f:
    lines = [x.decode('utf-8') for x in f.readlines()]    
    nid2name = dict([(l.split(" ")[0], l.split(" ")[2][:-1]) for l in lines])
    
dataset = datasets.ImageFolder(train_dir)
nid2idx = dataset.class_to_idx
idx2nid = {v: k for k, v in nid2idx.items()}
name2nid = {v: k for k, v in nid2name.items()}
idx2name = {k: nid2name[v] for k, v in idx2nid.items()}

## Analyze the train set on ImageNet

In [7]:
# CHANGE THIS TO CHANGE EXPERIMENT
# pyx_file = 'imagenet_val_out.npy' # NO FINE TUNING
pyx_file = 'imagenet__train__model_resnet50__pyx.npy' # trained from scratch with 10fold cv

# where imagenet dataset is located
train_dir = '/datasets/datasets/imagenet/train/'
# Stored results directory
pyx_dir = '/datasets/cgn/pyx/imagenet/'

# Load in data
pyx = np.load(pyx_dir + pyx_file)
imgs, labels = [list(z) for  z in zip(*datasets.ImageFolder(train_dir).imgs)]
labels = np.array(labels, dtype=int)

# A bad way to approach this problem might be to just look at the correlation of every column in the probability matrix. The problem is correlation is symmetric and this will correlate everything that has small probabilities. 

In [143]:
corr = np.corrcoef(pyx.T)

In [220]:
corr_non_diag = corr - np.eye(len(corr)) * corr.diagonal()
corr_largest_non_diag_raveled = np.argsort(corr_non_diag.ravel())[::-1]
corr_largest_non_diag = np.unravel_index(corr_largest_non_diag_raveled, corr_non_diag.shape)
corr_largest_non_diag = list(zip(*(list(z) for z in corr_largest_non_diag)))

print([(nid2name[idx2nid[z[0]]], nid2name[idx2nid[z[1]]]) for z in corr_largest_non_diag][:5])
print([nid2name[idx2nid[z]] for z in corr.diagonal().argsort()[:10]])

[('projectile', 'missile'), ('missile', 'projectile'), ('Eskimo_dog', 'Siberian_husky'), ('Siberian_husky', 'Eskimo_dog'), ('sunglass', 'sunglasses')]
['cleaver', 'Australian_terrier', 'miniature_poodle', 'weasel', 'titi', 'ballpoint', 'bow', 'breastplate', 'can_opener', 'CD_player']


# Understand ImageNet ontological issues on the TRAIN SET
## Uses ARGMAX baseline (not confident learning)

In [12]:
cj = confusion_matrix(np.argmax(pyx, axis=1), labels).T

In [13]:
joint = cleanlab.latent_estimation.estimate_joint(labels, pyx, cj)
joint_non_diag = joint - np.eye(len(joint)) * joint.diagonal()

In [14]:
cj_non_diag = cj - np.eye(len(cj)) * cj.diagonal()
largest_non_diag_raveled = np.argsort(cj_non_diag.ravel())[::-1]
largest_non_diag = np.unravel_index(largest_non_diag_raveled, cj_non_diag.shape)
largest_non_diag = list(zip(*(list(z) for z in largest_non_diag)))

In [15]:
# Checks that joint correctly has rows that are p(s)
assert(all(joint.sum(axis = 1) - np.bincount(labels) / len(labels) < 1e-4))

In [16]:
class_name = 'bighorn'

print("Index of '{}' in sorted diagonal of cj: ".format(class_name), end = "")
print([nid2name[idx2nid[i]] for i in cj.diagonal().argsort()].index(class_name))

print("Index of '{}' in sorted diagonal of joint: ".format(class_name), end = "")
print([nid2name[idx2nid[i]] for i in joint.diagonal().argsort()].index(class_name))

print("Index of '{}' in sorted most noisy classes in cj: ".format(class_name), end = "")
print([nid2name[idx2nid[i]] for i in np.argsort(cj_non_diag.sum(axis = 0))[::-1]].index(class_name))

print("Index of '{}' in sorted most noisy classes in joint: ".format(class_name), end = "")
print([nid2name[idx2nid[i]] for i in np.argsort(joint_non_diag.sum(axis = 0))[::-1]].index(class_name))

print("Index of '{}' in sorted most noisy true classes in cj: ".format(class_name), end = "")
print([nid2name[idx2nid[i]] for i in np.argsort(cj_non_diag.sum(axis = 1))[::-1]].index(class_name))

print("Index of '{}' in sorted most noisy true classes in joint: ".format(class_name), end = "")
print([nid2name[idx2nid[i]] for i in np.argsort(joint_non_diag.sum(axis = 1))[::-1]].index(class_name))

idx = cj.diagonal().argmin()
print("Least confident class by diagonal of cj:", nid2name[idx2nid[idx]], idx)
idx = joint.diagonal().argmin()
print("Least confident class by diagonal of joint:", nid2name[idx2nid[idx]], idx)
idx = cj_non_diag.sum(axis = 0).argmax()
print("Least confident class by max sum of row of non-diagonal elements of cj:", nid2name[idx2nid[idx]], idx)
idx = joint_non_diag.sum(axis = 1).argmax()
print("Least confident class by max sum of column of non-diagonal elements of cj:", nid2name[idx2nid[idx]], idx)
print('Largest noise rate:', [(nid2name[idx2nid[z]], z) for z in largest_non_diag[0]])

Index of 'bighorn' in sorted diagonal of cj: 573
Index of 'bighorn' in sorted diagonal of joint: 573
Index of 'bighorn' in sorted most noisy classes in cj: 237
Index of 'bighorn' in sorted most noisy classes in joint: 237
Index of 'bighorn' in sorted most noisy true classes in cj: 543
Index of 'bighorn' in sorted most noisy true classes in joint: 543
Least confident class by diagonal of cj: water_jug 899
Least confident class by diagonal of joint: water_jug 899
Least confident class by max sum of row of non-diagonal elements of cj: desk 526
Least confident class by max sum of column of non-diagonal elements of cj: water_jug 899
Largest noise rate: [('projectile', 744), ('missile', 657)]


In [17]:
cj

array([[1107,    5,    1, ...,    0,    0,    0],
       [   5, 1146,    0, ...,    0,    0,    0],
       [   0,    0,  994, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ...,  973,    0,    0],
       [   0,    0,    0, ...,    0,  716,    0],
       [   0,    0,    0, ...,    1,    0,  635]])

In [18]:
edges = [(
    idx2name[i].replace('American_chameleon', 'chameleon').replace('typewriter_keyboard', 'keyboard'), 
    idx2name[j].replace('American_chameleon', 'chameleon').replace('typewriter_keyboard', 'keyboard'), 
    idx2nid[i], 
    idx2nid[j], 
    int(round(cj[i,j])),
    joint[i,j].round(6),
) for i,j in largest_non_diag[:30]]
# nodes = list({z for i,j in largest_non_diag[:30] for z in (idx2name[i], idx2name[j])})

In [19]:
df = pd.DataFrame(edges)

In [25]:
df[r"$\tilde{y}$ name"].isin(['a','b'])

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
Name: $\tilde{y}$ name, dtype: bool

In [29]:
df[df[r"$\tilde{y}$ name"].isin(['projectile','tub', 'breastplate', 'chameleon', 'green_lizard', 'maillot', 'ram', 'missile', 'corn', 'keyboard'])][r"$C(\tilde{y},y^*)$"]

0     494
1     400
2     398
4     369
5     362
6     362
7     338
8     336
10    333
16    293
Name: $C(\tilde{y},y^*)$, dtype: int64

In [21]:
df = pd.DataFrame(edges, columns = [r"$\tilde{y}$ name", r"$y^*$ name", r"$\tilde{y}$ nid", r"$y^*$ nid", r"$C(\tilde{y},y^*)$", r"$P(\tilde{y},y^*)$"])[:20]
df.insert(loc = 0, column = 'Rank', value = df.index + 1)
tex = df.to_latex(index = False)
orig = '\\$\\textbackslash tilde\\{y\\}\\$ name &    \\$y\\textasciicircum *\\$ name & \\$\\textbackslash tilde\\{y\\}\\$ nid &  \\$y\\textasciicircum *\\$ nid &  \\$C(\\textbackslash tilde\\{y\\},y\\textasciicircum *)\\$ &  \\$P(\\textbackslash tilde\\{y\\},y\\textasciicircum *)\\$'
new = '$\\tilde{y}$ name   &   $y^*$ name   &   $\\tilde{y}$ nid   &   $y^*$ nid   &   $C(\\tilde{y},y^*)$   &   $P(\\tilde{y},y^*)$ '
tex = tex.replace(orig, new)
print(tex)
df.style.set_properties(subset=[r"$C(\tilde{y},y^*)$"], **{'width': '50px'})

\begin{tabular}{rllllrr}
\toprule
 Rank & \$\textbackslash tilde\{y\}\$ name &                    \$y\textasciicircum *\$ name & \$\textbackslash tilde\{y\}\$ nid &  \$y\textasciicircum *\$ nid &  \$C(\textbackslash tilde\{y\},y\textasciicircum *)\$ &  \$P(\textbackslash tilde\{y\},y\textasciicircum *)\$ \\
\midrule
    1 &       projectile &                       missile &       n04008634 &  n03773504 &                 494 &            0.000386 \\
    2 &              tub &                       bathtub &       n04493381 &  n02808440 &                 400 &            0.000312 \\
    3 &      breastplate &                       cuirass &       n02895154 &  n03146219 &                 398 &            0.000311 \\
    4 &    garden\_spider &  black\_and\_gold\_garden\_spider &       n01773797 &  n01773157 &                 377 &            0.000294 \\
    5 &        chameleon &                  green\_lizard &       n01682714 &  n01693334 &                 369 &            0.000288 \\
 

,Rank,$\tilde{y}$ name,$y^*$ name,$\tilde{y}$ nid,$y^*$ nid,"$C(\tilde{y},y^*)$","$P(\tilde{y},y^*)$"
0,1,projectile,missile,n04008634,n03773504,494,0.000386
1,2,tub,bathtub,n04493381,n02808440,400,0.000312
2,3,breastplate,cuirass,n02895154,n03146219,398,0.000311
3,4,garden_spider,black_and_gold_garden_spider,n01773797,n01773157,377,0.000294
4,5,chameleon,green_lizard,n01682714,n01693334,369,0.000288
5,6,green_lizard,chameleon,n01693334,n01682714,362,0.000283
6,7,maillot,maillot,n03710637,n03710721,362,0.000283
7,8,ram,bighorn,n02412080,n02415577,338,0.000264
8,9,missile,projectile,n03773504,n04008634,336,0.000262
9,10,horned_viper,sidewinder,n01753488,n01756291,335,0.000261


# Understand ImageNet ontological issues on the TRAIN SET

In [9]:
cj = cleanlab.latent_estimation.compute_confident_joint(labels, pyx)
joint = cleanlab.latent_estimation.estimate_joint(labels, pyx, cj)
joint_non_diag = joint - np.eye(len(joint)) * joint.diagonal()

In [10]:
cj_non_diag = cj - np.eye(len(cj)) * cj.diagonal()
largest_non_diag_raveled = np.argsort(cj_non_diag.ravel())[::-1]
largest_non_diag = np.unravel_index(largest_non_diag_raveled, cj_non_diag.shape)
largest_non_diag = list(zip(*(list(z) for z in largest_non_diag)))

In [11]:
# Checks that joint correctly has rows that are p(s)
assert(all(joint.sum(axis = 1) - np.bincount(labels) / len(labels) < 1e-4))

In [12]:
class_name = 'bighorn'

print("Index of '{}' in sorted diagonal of cj: ".format(class_name), end = "")
print([nid2name[idx2nid[i]] for i in cj.diagonal().argsort()].index(class_name))

print("Index of '{}' in sorted diagonal of joint: ".format(class_name), end = "")
print([nid2name[idx2nid[i]] for i in joint.diagonal().argsort()].index(class_name))

print("Index of '{}' in sorted most noisy classes in cj: ".format(class_name), end = "")
print([nid2name[idx2nid[i]] for i in np.argsort(cj_non_diag.sum(axis = 0))[::-1]].index(class_name))

print("Index of '{}' in sorted most noisy classes in joint: ".format(class_name), end = "")
print([nid2name[idx2nid[i]] for i in np.argsort(joint_non_diag.sum(axis = 0))[::-1]].index(class_name))

print("Index of '{}' in sorted most noisy true classes in cj: ".format(class_name), end = "")
print([nid2name[idx2nid[i]] for i in np.argsort(cj_non_diag.sum(axis = 1))[::-1]].index(class_name))

print("Index of '{}' in sorted most noisy true classes in joint: ".format(class_name), end = "")
print([nid2name[idx2nid[i]] for i in np.argsort(joint_non_diag.sum(axis = 1))[::-1]].index(class_name))

idx = cj.diagonal().argmin()
print("Least confident class by diagonal of cj:", nid2name[idx2nid[idx]], idx)
idx = joint.diagonal().argmin()
print("Least confident class by diagonal of joint:", nid2name[idx2nid[idx]], idx)
idx = cj_non_diag.sum(axis = 0).argmax()
print("Least confident class by max sum of row of non-diagonal elements of cj:", nid2name[idx2nid[idx]], idx)
idx = joint_non_diag.sum(axis = 1).argmax()
print("Least confident class by max sum of column of non-diagonal elements of cj:", nid2name[idx2nid[idx]], idx)
print('Largest noise rate:', [(nid2name[idx2nid[z]], z) for z in largest_non_diag[0]])

Index of 'bighorn' in sorted diagonal of cj: 374
Index of 'bighorn' in sorted diagonal of joint: 374
Index of 'bighorn' in sorted most noisy classes in cj: 149
Index of 'bighorn' in sorted most noisy classes in joint: 150
Index of 'bighorn' in sorted most noisy true classes in cj: 325
Index of 'bighorn' in sorted most noisy true classes in joint: 325
Least confident class by diagonal of cj: English_foxhound 167
Least confident class by diagonal of joint: English_foxhound 167
Least confident class by max sum of row of non-diagonal elements of cj: maillot 638
Least confident class by max sum of column of non-diagonal elements of cj: maillot 638
Largest noise rate: [('projectile', 744), ('missile', 657)]


In [13]:
cj

array([[1239,    3,    0, ...,    0,    0,    0],
       [   2, 1265,    0, ...,    0,    0,    0],
       [   0,    0, 1150, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ..., 1137,    0,    0],
       [   0,    0,    0, ...,    0,  874,    0],
       [   0,    0,    0, ...,    0,    0,  861]])

In [14]:
edges = [(
    idx2name[i].replace('American_chameleon', 'chameleon').replace('typewriter_keyboard', 'keyboard'), 
    idx2name[j].replace('American_chameleon', 'chameleon').replace('typewriter_keyboard', 'keyboard'), 
    idx2nid[i], 
    idx2nid[j], 
    int(round(cj[i,j])),
    joint[i,j].round(6),
) for i,j in largest_non_diag[:30]]
# nodes = list({z for i,j in largest_non_diag[:30] for z in (idx2name[i], idx2name[j])})

In [3]:
df = pd.DataFrame(edges)

NameError: name 'edges' is not defined

In [24]:
df = pd.DataFrame(edges, columns = [r"$\tilde{y}$ name", r"$y^*$ name", r"$\tilde{y}$ nid", r"$y^*$ nid", r"$C(\tilde{y},y^*)$", r"$P(\tilde{y},y^*)$"])[:10]
df.insert(loc = 0, column = 'Rank', value = df.index + 1)
tex = df.to_latex(index = False)
orig = '\\$\\textbackslash tilde\\{y\\}\\$ name &    \\$y\\textasciicircum *\\$ name & \\$\\textbackslash tilde\\{y\\}\\$ nid &  \\$y\\textasciicircum *\\$ nid &  \\$C(\\textbackslash tilde\\{y\\},y\\textasciicircum *)\\$ &  \\$P(\\textbackslash tilde\\{y\\},y\\textasciicircum *)\\$'
new = '$\\tilde{y}$ name   &   $y^*$ name   &   $\\tilde{y}$ nid   &   $y^*$ nid   &   $C(\\tilde{y},y^*)$   &   $P(\\tilde{y},y^*)$ '
tex = tex.replace(orig, new)
print(tex)
df.style.set_properties(subset=[r"$C(\tilde{y},y^*)$"], **{'width': '50px'})

\begin{tabular}{rllllrr}
\toprule
 Rank & $\tilde{y}$ name   &   $y^*$ name   &   $\tilde{y}$ nid   &   $y^*$ nid   &   $C(\tilde{y},y^*)$   &   $P(\tilde{y},y^*)$  \\
\midrule
    1 &       projectile &       missile &       n04008634 &  n03773504 &                 645 &            0.000503 \\
    2 &              tub &       bathtub &       n04493381 &  n02808440 &                 539 &            0.000421 \\
    3 &      breastplate &       cuirass &       n02895154 &  n03146219 &                 476 &            0.000372 \\
    4 &     green\_lizard &     chameleon &       n01693334 &  n01682714 &                 437 &            0.000341 \\
    5 &        chameleon &  green\_lizard &       n01682714 &  n01693334 &                 435 &            0.000340 \\
    6 &          missile &    projectile &       n03773504 &  n04008634 &                 433 &            0.000338 \\
    7 &          maillot &       maillot &       n03710637 &  n03710721 &                 417 &            

,Rank,$\tilde{y}$ name,$y^*$ name,$\tilde{y}$ nid,$y^*$ nid,"$C(\tilde{y},y^*)$","$P(\tilde{y},y^*)$"
0,1,projectile,missile,n04008634,n03773504,645,0.000503
1,2,tub,bathtub,n04493381,n02808440,539,0.000421
2,3,breastplate,cuirass,n02895154,n03146219,476,0.000372
3,4,green_lizard,chameleon,n01693334,n01682714,437,0.000341
4,5,chameleon,green_lizard,n01682714,n01693334,435,0.00034
5,6,missile,projectile,n03773504,n04008634,433,0.000338
6,7,maillot,maillot,n03710637,n03710721,417,0.000325
7,8,horned_viper,sidewinder,n01753488,n01756291,416,0.000325
8,9,corn,ear,n12144580,n13133613,410,0.00032
9,10,keyboard,space_bar,n04505470,n04264628,407,0.000318


In [257]:
tex

'\\begin{tabular}{lllllrr}\n\\toprule\n{} & \\$\\textbackslash tilde\\{y\\}\\$ name &    \\$y\\textasciicircum *\\$ name & \\$\\textbackslash tilde\\{y\\}\\$ nid &  \\$y\\textasciicircum *\\$ nid &  \\$C(\\textbackslash tilde\\{y\\},y\\textasciicircum *)\\$ &  \\$P(\\textbackslash tilde\\{y\\},y\\textasciicircum *)\\$ \\\\\n\\midrule\n0 &       projectile &       missile &       n04008634 &  n03773504 &                 645 &            0.000503 \\\\\n1 &              tub &       bathtub &       n04493381 &  n02808440 &                 539 &            0.000421 \\\\\n2 &      breastplate &       cuirass &       n02895154 &  n03146219 &                 476 &            0.000371 \\\\\n3 &     green\\_lizard &     chameleon &       n01693334 &  n01682714 &                 437 &            0.000341 \\\\\n4 &        chameleon &  green\\_lizard &       n01682714 &  n01693334 &                 435 &            0.000340 \\\\\n5 &          missile &    projectile &       n03773504 &  n04008634 &

In [250]:
for i,j in largest_non_diag[:30]:
    print(int(round(cj[i,j])), "|", idx2nid[i], idx2name[i], "|",  idx2nid[j], idx2name[j])

645 | n04008634 projectile | n03773504 missile
539 | n04493381 tub | n02808440 bathtub
476 | n02895154 breastplate | n03146219 cuirass
437 | n01693334 green_lizard | n01682714 American_chameleon
435 | n01682714 American_chameleon | n01693334 green_lizard
433 | n03773504 missile | n04008634 projectile
417 | n03710637 maillot | n03710721 maillot
416 | n01753488 horned_viper | n01756291 sidewinder
410 | n12144580 corn | n13133613 ear
406 | n04505470 typewriter_keyboard | n04264628 space_bar
399 | n04355933 sunglass | n04356056 sunglasses
389 | n02110185 Siberian_husky | n02109961 Eskimo_dog
370 | n02979186 cassette_player | n04392985 tape_player
369 | n04356056 sunglasses | n04355933 sunglass
367 | n02109961 Eskimo_dog | n02110185 Siberian_husky
364 | n01797886 ruffed_grouse | n01807496 partridge
360 | n02412080 ram | n02415577 bighorn
360 | n13133613 ear | n12144580 corn
354 | n02395406 hog | n02396427 wild_boar
354 | n03642806 laptop | n03832673 notebook
351 | n03146219 cuirass | n02895

In [129]:
print("Top 30 row sums in confident joint (most noisy classes):\n")
[(idx2nid[i], idx2name[i]) for i in np.argsort(cj_non_diag.sum(axis = 0))[::-1][:30]]

Top 30 row sums in confident joint (most noisy classes):



[('n04560804', 'water_jug'),
 ('n03710637', 'maillot'),
 ('n04392985', 'tape_player'),
 ('n09332890', 'lakeside'),
 ('n03180011', 'desktop_computer'),
 ('n02808440', 'bathtub'),
 ('n03832673', 'notebook'),
 ('n04041544', 'radio'),
 ('n09428293', 'seashore'),
 ('n03179701', 'desk'),
 ('n03773504', 'missile'),
 ('n02123045', 'tabby'),
 ('n01756291', 'sidewinder'),
 ('n04026417', 'purse'),
 ('n07579787', 'plate'),
 ('n04356056', 'sunglasses'),
 ('n02109961', 'Eskimo_dog'),
 ('n03976657', 'pole'),
 ('n03782006', 'monitor'),
 ('n04008634', 'projectile'),
 ('n03866082', 'overskirt'),
 ('n03871628', 'packet'),
 ('n03532672', 'hook'),
 ('n04152593', 'screen'),
 ('n01693334', 'green_lizard'),
 ('n01740131', 'night_snake'),
 ('n04355933', 'sunglass'),
 ('n03216828', 'dock'),
 ('n07930864', 'cup'),
 ('n02988304', 'CD_player')]

# Analyze the validation set on ImageNet

In [5]:
# CHANGE THIS TO CHANGE EXPERIMENT
# pyx_file = 'imagenet_val_out.npy' # NO FINE TUNING
pyx_file = 'imagenet_val_out_cv_10fold.npy' # fine tuned with 10fold cv

# where imagenet dataset is located
val_dir = '/datasets/datasets/imagenet/val/'
# Stored results directory
pyx_dir = '/datasets/cgn/pyx/imagenet/'

# Load in data
with open(pyx_dir + 'imagenet_val_out_cv_10fold.npy', 'rb') as f:
    out = np.load(f)
with open(pyx_dir + 'imagenet_val_labels.npy', 'rb') as f:
    labels = np.load(f)
pyx = torch.nn.functional.softmax(torch.from_numpy(out), dim = 1).numpy()

In [7]:
# set up mapping for imagenet validation data
url = 'https://gist.githubusercontent.com/aaronpolhamus/964a4411c0906315deb9f4a3723aac57/'
url += 'raw/aa66dd9dbf6b56649fa3fab83659b2acbf3cbfd1/map_clsloc.txt'
with urlopen(url) as f:
    lines = [x.decode('utf-8') for x in f.readlines()]    
    nid2name = dict([(l.split(" ")[0], l.split(" ")[2][:-1]) for l in lines])
    
dataset = datasets.ImageFolder(val_dir)
nid2idx = dataset.class_to_idx
idx2nid = {v: k for k, v in nid2idx.items()}
name2nid = {v: k for k, v in nid2name.items()}
idx2name = {k: nid2name[v] for k, v in idx2nid.items()}

In [8]:
cj = cleanlab.latent_estimation.estimate_confident_joint_from_probabilities(labels, pyx)
py, nm, inv = cleanlab.latent_estimation.estimate_latent(cj, labels)

In [9]:
cj_non_diag = cj - np.eye(len(cj)) * cj.diagonal()
largest_non_diag_raveled = np.argsort(cj_non_diag.ravel())[::-1]
largest_non_diag = np.unravel_index(largest_non_diag_raveled, cj_non_diag.shape)
largest_non_diag = list(zip(*(list(z) for z in largest_non_diag)))

In [39]:
print("Top 30 row sums in confident joint (most noisy classes):\n")
[(idx2nid[i], idx2name[i]) for i in np.argsort(cj_non_diag.sum(axis = 1))[::-1][:30]]

Top 30 row sums in confident joint (most noisy classes):



[('n02123159', 'tiger_cat'),
 ('n04355933', 'sunglass'),
 ('n04152593', 'screen'),
 ('n02113624', 'toy_poodle'),
 ('n02979186', 'cassette_player'),
 ('n02808440', 'bathtub'),
 ('n03642806', 'laptop'),
 ('n03637318', 'lampshade'),
 ('n02988304', 'CD_player'),
 ('n02107908', 'Appenzeller'),
 ('n02871525', 'bookshop'),
 ('n01753488', 'horned_viper'),
 ('n01682714', 'American_chameleon'),
 ('n03180011', 'desktop_computer'),
 ('n02110185', 'Siberian_husky'),
 ('n02412080', 'ram'),
 ('n03710637', 'maillot'),
 ('n12144580', 'corn'),
 ('n02441942', 'weasel'),
 ('n07734744', 'mushroom'),
 ('n03179701', 'desk'),
 ('n01740131', 'night_snake'),
 ('n02106030', 'collie'),
 ('n01667778', 'terrapin'),
 ('n04008634', 'projectile'),
 ('n03950228', 'pitcher'),
 ('n02119022', 'red_fox'),
 ('n03710721', 'maillot'),
 ('n03782006', 'monitor'),
 ('n04560804', 'water_jug')]

In [127]:
for i,j in largest_non_diag[:30]:
    print(cj[i,j], "|", idx2nid[i], idx2name[i], "|",  idx2nid[j], idx2name[j])

644.983461962514 | n04008634 projectile | n03773504 missile
538.9486260454004 | n04493381 tub | n02808440 bathtub
475.6818181818183 | n02895154 breastplate | n03146219 cuirass
436.84210526315803 | n01693334 green_lizard | n01682714 American_chameleon
435.0343473994112 | n01682714 American_chameleon | n01693334 green_lizard
432.89205702647666 | n03773504 missile | n04008634 projectile
417.0893054024257 | n03710637 maillot | n03710721 maillot
415.62819203268646 | n01753488 horned_viper | n01756291 sidewinder
410.09463722397487 | n12144580 corn | n13133613 ear
406.25000000000006 | n04505470 typewriter_keyboard | n04264628 space_bar
399.1764705882354 | n04355933 sunglass | n04356056 sunglasses
389.451476793249 | n02110185 Siberian_husky | n02109961 Eskimo_dog
369.8744769874478 | n02979186 cassette_player | n04392985 tape_player
368.79432624113485 | n04356056 sunglasses | n04355933 sunglass
366.8049792531121 | n02109961 Eskimo_dog | n02110185 Siberian_husky
363.6563876651983 | n01797886 ruf